# 03 - Classification with Multinomial Naive Bayes
In this section, we will fit a Naive Bayes classifier on a training set of labeled tweets and then assess the model's performance on a hold-out set of test tweets.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_colwidth = 400

%matplotlib inline

In [2]:
data_directory = os.path.join('..','data','')
csv_path = os.path.join(data_directory, 'twitter_hate_speech.csv')
df_orig = pd.read_csv(csv_path, engine = 'python', delimiter = ',')

In [3]:
df_orig.columns = ['id', 'golden','state', 'trusted_judgements','last_judgment','is_hate_speech','confidence',
                   'created','orig_golden','orig_last_judgement','orig_trusted_judgements','orig_id','orig_state',
                   'updated','orig_is_hate_speech','is_hate_speech_gold','reason','confidence2','id2','text']

# Drop columns

In [6]:
df = df_orig.dropna(thresh = 68, axis = 'columns')
df = df.set_index('id')

In [7]:
df.head(3)

,golden,state,trusted_judgements,last_judgment,is_hate_speech,confidence,id2,text
id,,,,,,,,
853718217,True,golden,86,NaN,The tweet uses offensive language but not hate speech,0.6013,1.666196e+09,Warning: penny boards will make you a faggot
853718218,True,golden,92,NaN,The tweet contains hate speech,0.7227,4.295121e+08,Fuck dykes
853718219,True,golden,86,NaN,The tweet contains hate speech,0.5229,3.956238e+08,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandonernandez @bootyacid at least i dont look like jefree starr faggot


# Recode hate speech classifications

In [8]:
df = df[['text','is_hate_speech']]
categories = df.is_hate_speech.unique()
df['is_hate_speech'] = df['is_hate_speech'].replace(categories, [1,2,0])

# Things to clean up
- twitter handles
- emoticons
- urls
- newlines
- hashtags
- duplicate tweets

In [9]:
df.head()

,text,is_hate_speech
id,,
853718217,Warning: penny boards will make you a faggot,1
853718218,Fuck dykes,2
853718219,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandonernandez @bootyacid at least i dont look like jefree starr faggot,2
853718220,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkillah Is a fag"" jackie jealous"" Neeeee",2
853718221,@Zhugstubble You heard me bitch but any way I'm back th texas so wtf u talking about bitch ass nigga,1


# Generate giant text file with all tweets

In [10]:
import spacy
nlp = spacy.load('en')

In [10]:
import re

clean_tweet = lambda tweet: re.sub(r'(@\w+\s*)|#|&|\(|\)|\"|(https?://\S*)|(�\S*\d*)|(128\d{3})|(_*UNDEF)',
                      'TWITTER_HANDLE ', tweet)

In [11]:
import spacy
nlp = spacy.load('en')

In [12]:
def lemmatize(tweet):
    x = str()
    for token in nlp(tweet):
        x = ' '.join([x,token.lemma_])
    return x[1:]

In [13]:
df_clean = df.copy()
df_clean['text'] = df_clean['text'].apply(clean_tweet)
df_clean['text'] = df_clean['text'].apply(lemmatize)
df_clean.head(10)

,text,is_hate_speech
id,,
853718217,warning : penny board will make -PRON- a faggot,1
853718218,fuck dyke,2
853718219,twitter_handle twitter_handle twitter_handle twitter_handle twitter_handle at least i do not look like jefree starr faggot,2
853718220,twitter_handle twitter_handle : twitter_handle twitter_handle : twitter_handle be a fagtwitter_handle jackie jealoustwitter_handle neeeee,2
853718221,twitter_handle -PRON- hear -PRON- bitch but any way -PRON- be back th texas so wtf u talk about bitch ass nigga,1
853718222,"twitter_handle -PRON- a dirty terrorist and -PRON- religion be a fucking joke , -PRON- go around scream allah akbar do terrorist shit . dirty faggot .",2
853718223,rt twitter_handle : twitter_handle look like faggot ?,2
853718224,well -PRON- think -PRON- know actually rt twitter_handle : man why y' all do not tell -PRON- -PRON- be a dick rid ass faggot ? y'all not real twitter_handle twitter_handle twitter_handle ; twitter_handle twitter_handle twitter_handle ; twitter_handle twitter_handle twitter_handle ; twitter_handle twitter_handle twitter_handle ; twitter_handle twitter_handle twitter_handle ; twitter_handle twit...,2
853718225,"twitter_handle -PRON- know . -PRON- be a joke , faggot .",1


# Drop duplicates

In [14]:
df_clean = df_clean.drop_duplicates(subset = 'text')

In [15]:
with open('../data/corpus_mnb.txt','wt') as file_out:
    for tweet in df_clean.text:
        file_out.write(tweet + '\n')

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.grid_search import GridSearchCV

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [17]:
def evaluate(y, y_pred):
    print('Accuracy:', accuracy_score(y, y_pred))
    print('F1 weighted: ', f1_score(y, y_pred, average = 'weighted'))
    print('Confusion matrix:\n', confusion_matrix(y, y_pred))

In [18]:
X = df_clean.text
X_train, X_test, y_train, y_test = train_test_split(df_clean.text, df_clean.is_hate_speech,
                                                    test_size = 0.4, stratify = df_clean.is_hate_speech)

In [21]:
steps = [('vect', TfidfVectorizer()),('mnb', MultinomialNB())]
pipe = Pipeline(steps)

parameters = {'vect__min_df': np.arange(0,0.011, 0.002),
              'vect__max_df': np.arange(.99,1.001, 0.002),
             'mnb__alpha': np.arange(0,1.1,0.1)}

pipe_cv = GridSearchCV(pipe, param_grid = parameters, cv = 3, scoring = 'accuracy')

pipe_cv.fit(X_train, y_train)
y_train_pred = pipe_cv.predict(X_train)
y_test_pred = pipe_cv.predict(X_test)

In [22]:
evaluate(y_train_pred, y_train)

Accuracy: 0.85490844354
F1 weighted:  0.871051956447
Confusion matrix:
 [[3885  163  106]
 [ 170 2421  681]
 [   7   14  417]]


In [23]:
evaluate(y_test, y_test_pred)

Accuracy: 0.763348588863
F1 weighted:  0.725072909588
Confusion matrix:
 [[2483  214   12]
 [ 252 1443   37]
 [ 119  607   77]]


# What if we collapse offensive and hate speech into the same category?

In [27]:
y_binary = (y_test == 1) | (y_test == 2)
y_pred_binary = (y_test_pred == 1) | (y_test_pred == 2)
evaluate(y_binary, y_pred_binary)

Accuracy: 0.886155606407
F1 weighted:  0.885963405675
Confusion matrix:
 [[2483  226]
 [ 371 2164]]


In [76]:
def tweet_rater(tweet, clf):
    tweet = lemmatize(tweet)
    print(tweet)
    rating = clf.predict([tweet])
    probability = clf.predict_proba([tweet])
    print(probability)
    if rating == 0:
        print('I\'m {:2.4}% sure that\'s not offensive.'.format(probability[0][0]*100))
    elif rating == 1:
        print('I\'m {:2.4}% sure that\'s offensive.'.format(probability[0][1]*100))
    else:
        print('I\'m {:2.4}% sure that\'s hate speech.'.format(probability[0][2]*100))

In [93]:
tweet_rater('i hate when that happens', pipe_cv)

i hate when that happen
[[ 0.37662906  0.46452337  0.15884757]]
I'm 46.45% sure that's offensive.


In [26]:
vect = pipe_cv.best_estimator_.steps[0][1]
clf = pipe_cv.best_estimator_.steps[1][1]
X_train_vect = vect.fit_transform(X_train)
words = np.array(vect.get_feature_names())
x = np.eye(X_train_vect.shape[1])
probs = clf.predict_log_proba(x)
probs_hate = np.exp(probs[:,2])
df = pd.DataFrame(dict(words = words, probs_hate = probs_hate))
df.sort_values(by = 'probs_hate', ascending = False)

,probs_hate,words
312,0.754578,__
5756,0.711169,kys
2832,0.672331,dementia
4511,0.672331,gtfo
6653,0.672331,moon
4817,0.621425,hitler
2956,0.621425,dike
6769,0.621425,mussilm
9737,0.621425,sumeztwitter_handle
11271,0.621425,yas
